This notebook creates visualizations of the ChatGPT annotations in comparison to human annotations, for all the six NLP tasks.

* Input: 'AgoraSpeech_preprocessed.csv'
* Output: 'criticim_agenda_gpt.png', 'entities_gpt.png', 'polarization_gpt.png', 'populism_gpt.png', 'sentiment_gpt.png', and topics_gpt.png'
* Actions: It creates visualizations for for comparing the ChatGPT vs human annotations for the NLP tasks:
    1. Text classification: criticism vs. political agenda
    2. Topic classification: from a predefined list of topics (https://lab.imedd.org/en/pos-analyoume-tis-proeklogikes-omilies-ton-politikon-archigon/)
    3. Sentiment analysis: classification into negative, neutral, and positive
    4. Polarization detection: classification into low, medium, and high
    5. Populism detection: classification into low, medium, and high

In [1]:
import warnings
import pandas as pd
import plotly.express as px
warnings.filterwarnings('ignore')

In [2]:
# read data
data = pd.read_csv('AgoraSpeech_preprocessed.csv')

Text classification: criticism vs. political agenda

In [3]:
# prepare the data for the plot
df_melted = data[['criticism_or_agenda_gpt', 'criticism_or_agenda_human']].melt(var_name='feature', value_name='value')
df_grouped = df_melted.groupby(['feature', 'value']).size().reset_index(name='count')
totals = df_grouped.groupby('feature')['count'].sum().reset_index()
totals['value'] = 'total'

# plot the grouped bars
color_map = {'criticism_or_agenda_gpt': '#fdbf81', 'criticism_or_agenda_human': '#abd9e9'}
fig = px.bar(df_grouped, x='value', y='count', color='feature', text='count', barmode='group',
    labels={'count': 'Count of paragraphs', 'value': 'Criticism vs. Agenda'}, text_auto=True, color_discrete_map=color_map)
fig.update_traces(textposition='outside', textfont=dict(size=16, family='Arial'))

# add the lines for the totals
for index, row in totals.iterrows():
    fig.add_shape(
        type='line',
        x0=-0.45,  
        x1=1.35,
        y0=row['count'], y1=row['count'],
        xref='x', yref='y',
        line=dict(color=color_map[row['feature']], width=4, dash='dash'),
        name=f'Total {row["feature"]}')
    fig.add_annotation(
        x=1.35,
        y=row['count'] + (index * 100),
        text=f'Total: {row["count"]}',
        showarrow=False,
        font=dict(size=14, family='Arial', color='black'),
        xanchor='left',
        yanchor='middle')

# adjust legend and figure size
custom_labels = {'criticism_or_agenda_gpt': 'ChatGPT', 'criticism_or_agenda_human': 'Human'}
fig.for_each_trace(lambda t: t.update(name=custom_labels[t.name]))
fig.update_layout(
    width=700, height=500,
    legend_title_text='Annotations',
    legend=dict(orientation="h", yanchor="bottom", y=1.0, xanchor="center", x=0.5, font=dict(size=16, family='Arial')))

# adjust axis
fig.update_xaxes(title=None)
fig.update_xaxes(tickfont=dict(size=18, family='Arial', color='black'))
fig.update_yaxes(title_font=dict(size=18, family='Arial', color='black'))

fig.show()
fig.write_image("images/criticism_agenda_gpt.png", scale=2)

Topic classification: from a predefined list of topics (https://lab.imedd.org/en/pos-analyoume-tis-proeklogikes-omilies-ton-politikon-archigon/)

In [4]:
# prepare the data
topic_counts_human = data['topic_human'].value_counts().reset_index().rename(columns={'index': 'topic', 'topic_human': 'topic'})
topic_counts_human['column'] = 'topic_human'
topic_counts_gpt = data['topic_gpt'].value_counts().reset_index().rename(columns={'index': 'topic', 'topic_gpt': 'topic'})
topic_counts_gpt['column'] = 'topic_gpt'
df_counts = pd.concat([topic_counts_gpt, topic_counts_human], ignore_index=True)
top_5_topics = df_counts.groupby('topic')['count'].sum().nlargest(8).index
df_counts_filtered = df_counts[df_counts['topic'].isin(top_5_topics)]
df_counts_filtered['column'] = df_counts_filtered['column'].replace({'topic_gpt': 'ChatGPT', 'topic_human': 'Human'})
totals_full_df = pd.concat([topic_counts_human, topic_counts_gpt], ignore_index=True).groupby('column')['count'].sum().reset_index()

# plot the stacked bars
custom_colors = ['#253494', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4', '#edf8b1', '#ffffd9']
fig = px.bar(df_counts_filtered, x='column', y='count', color='topic', labels={'count': 'Count of paragraphs'},  color_discrete_sequence=custom_colors)

# add the lines for the sum
color_map = {'topic_gpt': '#fdbf81', 'topic_human': '#abd9e9'}
for i, row in totals_full_df.iterrows():
    fig.add_shape(
        type='line',
        x0=i - 0.4,
        x1=i + 0.4,
        y0=row['count'],
        y1=row['count'],
        line=dict(color=color_map[row['column']], width=4, dash='dash'))
    fig.add_annotation(
        x=i,
        y=row['count'] + 5,
        text=f'Total: {row["count"]}',
        showarrow=False,
        font=dict(size=14, family='Arial', color='black'),
        xanchor='center',
        yanchor='bottom')

# adjust legend and figure size
fig.update_layout(
    width=700, height=500,
    legend_title_text='Topics',
    legend=dict(orientation="h", yanchor="bottom", y=1.0, xanchor="center", x=0.5, font=dict(size=16, family='Arial')))

# adjust axis
fig.update_xaxes(title=None)
fig.update_xaxes(tickfont=dict(size=18, family='Arial', color='black'))
fig.update_yaxes(title_font=dict(size=18, family='Arial', color='black'))

fig.show()
fig.write_image("images/topics_gpt.png", scale=2)

Sentiment analysis: classification into negative, neutral, and positive

In [5]:
# prepare the data for the plot
df_melted = data[['sentiment_gpt_category', 'sentiment_human_category']].melt(var_name='feature', value_name='value')
df_grouped = df_melted.groupby(['feature', 'value']).size().reset_index(name='count')
totals = df_grouped.groupby('feature')['count'].sum().reset_index()
totals['value'] = 'total'

# plot the grouped bars
color_map = {'sentiment_gpt_category': '#fdbf81', 'sentiment_human_category': '#abd9e9'}
fig = px.bar(df_grouped, x='value', y='count', color='feature', text='count', barmode='group',
    labels={'count': 'Count of paragraphs', 'value': 'Sentiment'}, text_auto=True, color_discrete_map=color_map)
fig.update_traces(textposition='outside', textfont=dict(size=16, family='Arial'))

# add the lines for the totals
for index, row in totals.iterrows():
    fig.add_shape(
        type='line',
        x0=-0.45,  
        x1=2.35,
        y0=row['count'], y1=row['count'],
        xref='x', yref='y',
        line=dict(color=color_map[row['feature']], width=4, dash='dash'),
        name=f'Total {row["feature"]}')
    fig.add_annotation(
        x=2.35,
        y=row['count'],
        text=f'Total: {row["count"]}',
        showarrow=False,
        font=dict(size=14, family='Arial', color='black'),
        xanchor='left',
        yanchor='middle')

# adjust legend and figure size
custom_labels = {'sentiment_gpt_category': 'ChatGPT', 'sentiment_human_category': 'Human'}
fig.for_each_trace(lambda t: t.update(name=custom_labels[t.name]))
fig.update_layout(
    width=700, height=500,
    legend_title_text='Annotations',
    legend=dict(orientation="h", yanchor="bottom", y=1.0, xanchor="center", x=0.5, font=dict(size=16, family='Arial')))

# adjust axis
fig.update_xaxes(title=None)
fig.update_xaxes(tickfont=dict(size=18, family='Arial', color='black'))
fig.update_yaxes(title_font=dict(size=18, family='Arial', color='black'))

fig.show()
fig.write_image("images/sentiment_gpt.png", scale=2)

Polarization detection: classification into low, medium, and high

In [6]:
# prepare the data for the plot
df_melted = data[['polarization_gpt_category', 'polarization_human_category']].melt(var_name='feature', value_name='value')
df_grouped = df_melted.groupby(['feature', 'value']).size().reset_index(name='count')
totals = df_grouped.groupby('feature')['count'].sum().reset_index()
totals['value'] = 'total'

# plot the grouped bars
color_map = {'polarization_gpt_category': '#fdbf81', 'polarization_human_category': '#abd9e9'}
fig = px.bar(df_grouped, x='value', y='count', color='feature', text='count', barmode='group',
             category_orders={'value': ['low', 'medium', 'high']},
             labels={'count': 'Count of paragraphs', 'value': 'Sentiment'}, text_auto=True, color_discrete_map=color_map)
fig.update_traces(textposition='outside', textfont=dict(size=16, family='Arial'))

# add the lines for the totals
for index, row in totals.iterrows():
    fig.add_shape(
        type='line',
        x0=-0.45,  
        x1=2.35,
        y0=row['count'], y1=row['count'],
        xref='x', yref='y',
        line=dict(color=color_map[row['feature']], width=4, dash='dash'),
        name=f'Total {row["feature"]}')
    fig.add_annotation(
        x=2.35,
        y=row['count'] + (index * 150),
        text=f'Total: {row["count"]}',
        showarrow=False,
        font=dict(size=14, family='Arial', color='black'),
        xanchor='left',
        yanchor='middle')

# adjust legend and figure size
custom_labels = {'polarization_gpt_category': 'ChatGPT', 'polarization_human_category': 'Human'}
fig.for_each_trace(lambda t: t.update(name=custom_labels[t.name]))
fig.update_layout(
    width=700, height=500,
    legend_title_text='Annotations',
    legend=dict(orientation="h", yanchor="bottom", y=1.0, xanchor="center", x=0.5, font=dict(size=16, family='Arial')))

# adjust axis
fig.update_xaxes(title=None)
fig.update_xaxes(tickfont=dict(size=18, family='Arial', color='black'))
fig.update_yaxes(title_font=dict(size=18, family='Arial', color='black'))

fig.show()
fig.write_image("images/polarization_gpt.png", scale=2)

Populism detection: classification into low, medium, and high

In [7]:
# prepare the data for the plot
df_melted = data[['populism_gpt_category', 'populism_human_category']].melt(var_name='feature', value_name='value')
df_grouped = df_melted.groupby(['feature', 'value']).size().reset_index(name='count')
totals = df_grouped.groupby('feature')['count'].sum().reset_index()
totals['value'] = 'total'

# plot the grouped bars
color_map = {'populism_gpt_category': '#fdbf81', 'populism_human_category': '#abd9e9'}
fig = px.bar(df_grouped, x='value', y='count', color='feature', text='count', barmode='group',
             category_orders={'value': ['low', 'medium', 'high']},
             labels={'count': 'Count of paragraphs', 'value': 'Sentiment'}, text_auto=True, color_discrete_map=color_map)
fig.update_traces(textposition='outside', textfont=dict(size=16, family='Arial'))

# add the lines for the totals
for index, row in totals.iterrows():
    fig.add_shape(
        type='line',
        x0=-0.45,  
        x1=2.35,
        y0=row['count'], y1=row['count'],
        xref='x', yref='y',
        line=dict(color=color_map[row['feature']], width=4, dash='dash'),
        name=f'Total {row["feature"]}')
    fig.add_annotation(
        x=2.35,
        y=row['count'] + (index * 150),
        text=f'Total: {row["count"]}',
        showarrow=False,
        font=dict(size=14, family='Arial', color='black'),
        xanchor='left',
        yanchor='middle')

# adjust legend and figure size
custom_labels = {'populism_gpt_category': 'ChatGPT', 'populism_human_category': 'Human'}
fig.for_each_trace(lambda t: t.update(name=custom_labels[t.name]))
fig.update_layout(
    width=700, height=500,
    legend_title_text='Annotations',
    legend=dict(orientation="h", yanchor="bottom", y=1.0, xanchor="center", x=0.5, font=dict(size=16, family='Arial')))

# adjust axis
fig.update_xaxes(title=None)
fig.update_xaxes(tickfont=dict(size=18, family='Arial', color='black'))
fig.update_yaxes(title_font=dict(size=18, family='Arial', color='black'))

fig.show()
fig.write_image("images/populism_gpt.png", scale=2)